In [1]:
import sys
sys.path.insert(0, "/home/alexp/NBA_Models/src/scripts/")

In [2]:
from data.transform_data import transform
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3
import tensorflow as tf

%load_ext tensorboard

import datetime

pd.set_option("display.max_columns",None)

2024-02-07 12:58:41.157473: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 12:58:41.157508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 12:58:41.158050: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-07 12:58:41.161186: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-07 12:58:41.587845: W tensorflow/compiler/tf2

In [8]:
path = "/home/alexp/NBA_Models/sqlite/db/nba_data.db"
conn = sqlite3.connect(path)

t = transform(conn,2010,2020)
clean = t.transform(type='team_boxscores')
lable_enc = t.label_encoder(clean,['SEASON','TEAM_ABBREVIATION'])
pct_change = t.convert_pct_change(lable_enc,type='multi')
df = lable_enc

test = transform(conn,2021,2023)
test_clean = test.transform(type='agg_boxscores')
test_scaler = test.standard_scalar(test_clean)
test_lable_enc = test.label_encoder(test_scaler,['SEASON','TEAM_ABBREVIATION'])
test_data = test_lable_enc

/home/alexp/NBA_Models/src/scripts/data/transform_data.py:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team[filter] = team[filter].pct_change(axis='index',periods=1)
/home/alexp/NBA_Models/src/scripts/data/transform_data.py:233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team[filter] = team[filter].replace([np.inf,-np.inf], 0)
/home/alexp/NBA_Models/src/scripts/data/transform_data.py:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [9]:
pct_change

,level_0,index,SEASON,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,HOME_GAME,WL,FG2M,FG2A,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,POSS,PIE,PTS_2PT_MR,PTS_FB,PTS_OFF_TOV,PTS_PAINT,AST_2PM,AST_3PM,UAST_2PM,UAST_3PM
0,0,43,0,1610612737,0,Atlanta Hawks,0021300023,2013-11-01,ATL vs. TOR,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,112,0,1610612737,0,Atlanta Hawks,0021300059,2013-11-05,ATL @ SAC,0,1,0.192308,0.092593,-0.200000,-0.043478,-0.050000,-0.161290,0.285714,-0.100000,-0.027027,0.083333,0.571429,1.333333,0.076923,0.250000,0.029412,-0.285714,0.021801,0.018622,0.040486,0.031000,-0.279412,-0.148649,0.010526,-0.047700,0.083333,0.571429,0.833333,0.205128,0.384615,-0.200000,0.000000,NaN
2,2,141,0,1610612737,0,Atlanta Hawks,0021300071,2013-11-07,ATL @ DEN,0,0,0.129032,0.152542,0.000000,0.181818,-0.315789,-0.269231,0.444444,0.222222,0.277778,0.192308,0.000000,0.285714,-0.214286,0.533333,0.019048,-1.400000,-0.011132,-0.001828,0.066148,0.090204,-1.612245,-1.507937,0.020833,-0.101968,0.307692,0.090909,-0.090909,0.106383,0.333333,-0.125000,-0.166667,0.0
3,3,184,0,1610612737,0,Atlanta Hawks,0021300086,2013-11-09,ATL vs. ORL,1,1,0.057143,0.044118,-0.250000,-0.269231,-0.076923,-0.105263,-0.461538,-0.030303,-0.152174,0.161290,-0.181818,-0.333333,0.181818,-0.347826,-0.028037,-6.000000,-0.057223,-0.028388,-0.133212,-0.146797,-2.833333,-4.187500,0.000000,0.252988,1.823529,0.083333,-0.350000,-0.500000,0.250000,-0.142857,-0.400000,-1.0
4,4,194,0,1610612737,0,Atlanta Hawks,0021300098,2013-11-11,ATL @ CHA,0,1,-0.324324,-0.309859,0.666667,0.421053,0.916667,0.647059,-0.142857,0.093750,0.051282,-0.277778,-0.222222,-0.500000,0.000000,0.466667,-0.009615,-0.100000,0.075622,0.011310,0.012632,0.042753,1.163636,-0.284314,-0.020408,-0.046105,-0.625000,-0.692308,0.230769,0.230769,-0.433333,0.500000,0.333333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17413,565,20216,7,1610612764,29,Washington Wizards,0042000101,2021-05-23,WAS @ PHI,0,0,0.205882,0.133333,-0.200000,-0.354839,-0.294118,-0.285714,-0.300000,-0.209302,-0.226415,0.181818,0.000000,0.333333,0.142857,0.200000,0.026087,-2.400000,0.032638,0.026619,0.158358,0.170132,-1.567901,-2.157143,0.000000,-0.076190,-0.666667,2.600000,-0.428571,0.500000,0.285714,-0.125000,0.157895,-0.5
17414,566,20230,7,1610612764,29,Washington Wizards,0042000102,2021-05-26,WAS @ PHI,0,0,-0.146341,0.029412,-0.750000,0.100000,0.583333,1.000000,0.857143,-0.205882,-0.024390,-0.153846,-0.250000,-0.500000,-0.125000,-0.125000,-0.194915,2.571429,-0.214223,-0.153846,0.009283,-0.021002,5.543478,1.876543,-0.049020,-0.385567,2.333333,-0.388889,-0.125000,-0.346667,0.055556,-0.714286,-0.318182,-1.0
17415,567,20250,7,1610612764,29,Washington Wizards,0042000103,2021-05-29,WAS vs. PHI,1,0,-0.142857,-0.128571,3.000000,0.590909,0.000000,-0.166667,0.000000,0.037037,0.025000,-0.181818,0.666667,0.500000,-0.428571,-0.142857,0.084211,0.160000,0.128492,0.021450,0.053512,0.078383,-0.169435,0.317597,0.061856,0.161074,-0.100000,0.090909,1.142857,-0.142857,-0.368421,2.000000,0.200000,0.0
17416,568,20262,7,1610612764,29,Washington Wizards,0042000104,2021-05-31,WAS vs. PHI,1,1,0.033333,0.049180,0.125000,-0.314286,0.736842,0.680000,0.000000,0.571429,0.390244,0.333333,0.600000,3.000000,0.750000,0.500000,0.184466,-1.275862,0.123762,0.109000,-0.176984,-0.192043,-1.392000,-1.175896,0.067961,0.708092,-0.222222,0.666667,0.000000,0.119048,0.250000,0.500000,-0.166667,-1.0


In [10]:
pct_change = pct_change.loc[2:,:].drop(columns=['TEAM_ID','TEAM_NAME','GAME_ID','GAME_DATE','MATCHUP','UAST_3PM','PTS_2PT_MR','level_0','index'])

In [11]:
pct_change

,SEASON,TEAM_ABBREVIATION,HOME_GAME,WL,FG2M,FG2A,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,POSS,PIE,PTS_FB,PTS_OFF_TOV,PTS_PAINT,AST_2PM,AST_3PM,UAST_2PM
3,0,0,1,1,0.057143,0.044118,-0.250000,-0.269231,-0.076923,-0.105263,-0.461538,-0.030303,-0.152174,0.161290,-0.181818,-0.333333,0.181818,-0.347826,-0.028037,-6.000000,-0.057223,-0.028388,-0.133212,-0.146797,-2.833333,-4.187500,0.000000,0.252988,0.083333,-0.350000,-0.500000,0.250000,-0.142857,-0.400000
4,0,0,0,1,-0.324324,-0.309859,0.666667,0.421053,0.916667,0.647059,-0.142857,0.093750,0.051282,-0.277778,-0.222222,-0.500000,0.000000,0.466667,-0.009615,-0.100000,0.075622,0.011310,0.012632,0.042753,1.163636,-0.284314,-0.020408,-0.046105,-0.692308,0.230769,0.230769,-0.433333,0.500000,0.333333
5,0,0,1,1,0.160000,0.183673,0.000000,-0.148148,0.086957,0.107143,0.000000,0.057143,0.048780,-0.038462,-0.142857,0.666667,0.230769,0.045455,0.097087,0.111111,-0.000925,0.022367,0.038462,0.010000,-0.319328,0.191781,0.072917,-0.078333,4.000000,0.000000,0.500000,-0.058824,0.000000,0.500000
6,0,0,0,0,-0.241379,-0.137931,0.100000,0.043478,-0.560000,-0.516129,1.166667,-0.297297,-0.093023,-0.120000,0.333333,-0.400000,0.500000,-0.086957,-0.221239,-2.600000,-0.110185,-0.137648,0.092092,0.106931,-2.604938,-2.977011,-0.097087,-0.294756,-0.350000,0.125000,-0.208333,-0.312500,0.000000,-0.166667
7,0,0,1,1,0.454545,0.120000,-0.636364,-0.375000,0.545455,0.600000,-0.615385,0.269231,-0.025641,0.045455,0.375000,1.666667,-0.083333,-0.428571,0.056818,-1.500000,-0.017690,0.045455,-0.173236,-0.182469,-1.323077,-1.436047,0.010753,0.541026,-0.230769,-0.444444,0.315789,0.727273,-0.555556,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17413,7,29,0,0,0.205882,0.133333,-0.200000,-0.354839,-0.294118,-0.285714,-0.300000,-0.209302,-0.226415,0.181818,0.000000,0.333333,0.142857,0.200000,0.026087,-2.400000,0.032638,0.026619,0.158358,0.170132,-1.567901,-2.157143,0.000000,-0.076190,2.600000,-0.428571,0.500000,0.285714,-0.125000,0.157895
17414,7,29,0,0,-0.146341,0.029412,-0.750000,0.100000,0.583333,1.000000,0.857143,-0.205882,-0.024390,-0.153846,-0.250000,-0.500000,-0.125000,-0.125000,-0.194915,2.571429,-0.214223,-0.153846,0.009283,-0.021002,5.543478,1.876543,-0.049020,-0.385567,-0.388889,-0.125000,-0.346667,0.055556,-0.714286,-0.318182
17415,7,29,1,0,-0.142857,-0.128571,3.000000,0.590909,0.000000,-0.166667,0.000000,0.037037,0.025000,-0.181818,0.666667,0.500000,-0.428571,-0.142857,0.084211,0.160000,0.128492,0.021450,0.053512,0.078383,-0.169435,0.317597,0.061856,0.161074,0.090909,1.142857,-0.142857,-0.368421,2.000000,0.200000
17416,7,29,1,1,0.033333,0.049180,0.125000,-0.314286,0.736842,0.680000,0.000000,0.571429,0.390244,0.333333,0.600000,3.000000,0.750000,0.500000,0.184466,-1.275862,0.123762,0.109000,-0.176984,-0.192043,-1.392000,-1.175896,0.067961,0.708092,0.666667,0.000000,0.119048,0.250000,0.500000,-0.166667


In [62]:
X, y = t.create_windows(df,30,type='single')

In [63]:
from model import ModelConfig

In [64]:
model_config = ModelConfig(X,y,[.75,.15,.10])
X_train,y_train,X_test,y_test,X_val,y_val = model_config.train_test_val()
X_train.shape

(452, 30, 34)

In [65]:
import keras
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.losses import MeanSquaredError,BinaryCrossentropy
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.metrics import RootMeanSquaredError, Accuracy
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

#,kernel_regularizer=regularizers.L2(0.0001)
model = model_config.create_sequential()
model.add(LSTM(256,'tanh',kernel_regularizer=regularizers.L2(0.0001)))
model.add(Dropout(.2))
model.add(Dense(64,'tanh',kernel_regularizer=regularizers.L2(0.0001)))
model.add(Dropout(.2))
model.add(Dense(32,'tanh',kernel_regularizer=regularizers.L2(0.0001)))
model.add(Dropout(.2))
model.add(Dense(1,'sigmoid'))

model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=.0001), metrics=[RootMeanSquaredError(),'accuracy'])

In [66]:

cp = ModelCheckpoint("/home/alexp/NBA_Models/models/models/model5", save_best_only=True)
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=100,callbacks=[tensorboard_callback,cp])

Epoch 1/100
13/15 [=========================>....] - ETA: 0s - loss: 0.4509 - root_mean_squared_error: 0.6558 - accuracy: 0.5337INFO:tensorflow:Assets written to: /home/alexp/NBA_Models/models/models/model5/assets


INFO:tensorflow:Assets written to: /home/alexp/NBA_Models/models/models/model5/assets


15/15 [==============================] - 7s 402ms/step - loss: 0.4512 - root_mean_squared_error: 0.6560 - accuracy: 0.5221 - val_loss: 0.2379 - val_root_mean_squared_error: 0.4663 - val_accuracy: 0.7000
Epoch 2/100
15/15 [==============================] - 8s 517ms/step - loss: 0.3859 - root_mean_squared_error: 0.6047 - accuracy: 0.5332 - val_loss: 0.2593 - val_root_mean_squared_error: 0.4892 - val_accuracy: 0.7000
Epoch 3/100
15/15 [==============================] - 1s 92ms/step - loss: 0.3734 - root_mean_squared_error: 0.5947 - accuracy: 0.5265 - val_loss: 0.2601 - val_root_mean_squared_error: 0.4905 - val_accuracy: 0.7000
Epoch 4/100
15/15 [==============================] - 7s 505ms/step - loss: 0.3837 - root_mean_squared_error: 0.6037 - accuracy: 0.5376 - val_loss: 0.2567 - val_root_mean_squared_error: 0.4877 - val_accuracy: 0.7000
Epoch 5/100
15/15 [==============================] - 3s 179ms/step - loss: 0.3820 - root_mean_squared_error: 0.6027 - accuracy: 0.5288 - val_loss: 0.2514

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("/home/alexp/NBA_Models/models/models/model5")

In [ ]:
model.evaluate(X_test,y_test)

3/3 [==============================] - 0s 5ms/step - loss: 0.2619 - root_mean_squared_error: 0.5041 - accuracy: 0.6154


[0.26191025972366333, 0.5041092038154602, 0.6153846383094788]

In [ ]:
preds_raw = model.predict(X_test)
actual = y_test
preds = []
for i,pred in enumerate(preds_raw):
    if pred[0] >=.5:
        preds.append(1)
    elif pred[0] <.5:
        preds.append(0)

confusion_matrix(actual,preds),accuracy_score(actual,preds), f1_score(actual,preds)

3/3 [==============================] - 0s 2ms/step


(array([[ 6, 34],
        [ 1, 50]]),
 0.6153846153846154,
 0.7407407407407407)